<a href="https://colab.research.google.com/github/vishalmysore/AI/blob/main/examples/Chroma_Example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install chromadb datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00


In [19]:
from datasets import load_dataset
dataset = load_dataset("medmcqa", split="train")

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [20]:
dataset[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract'}

In [22]:
import pandas as pd
df = pd.DataFrame(dataset)


In [23]:
df.isna().sum()
# Columns exp and topic_name contain missing values, we will drop rows with missing values in both columns
df = df.dropna(subset=["exp"])
df.isna().sum()
df = df.dropna(subset=["topic_name"])
df.isna().sum()

id              0
question        0
opa             0
opb             0
opc             0
opd             0
cop             0
choice_type     0
exp             0
subject_name    0
topic_name      0
dtype: int64

In [24]:
#converting the pre-processed dataframe back to dataset
dataset = dataset.from_pandas(df)

In [25]:
# Now import chromadb, instantiate a chroma client and create a collection using the function create_collection, give it a name.
# you can specify the embedding function to be used as a parameter in the create_collection(), if you don't, a default technique is used.
import chromadb
client = chromadb.Client()
collection = client.create_collection("MedicalQA")

In [26]:
# Now we will store the expert explanation field of first 10 questions from dataset into collection.
collection.add(
    ids=[str(i) for i in range(0, 10)],  # every entry in collection requires an ID, ID is a string
    documents=dataset["exp"][:10],# explanation field text, for first 10 questions only
    metadatas=[{"type": "exp"} for _ in range(0, 10)],) #metadata, additional information about the documents

In [27]:
collection.peek() # peek() returns a list of the first 10 items in the collection

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': [[0.017796462401747704,
   0.02271975763142109,
   0.07299382239580154,
   -0.025254027917981148,
   -0.05387657508254051,
   -0.059407368302345276,
   -0.026349546387791634,
   0.07464781403541565,
   0.02873391844332218,
   -0.026618750765919685,
   -0.049058668315410614,
   0.029021840542554855,
   0.0375068224966526,
   0.008624397218227386,
   -0.06284338235855103,
   -0.05127057805657387,
   -0.0997946485877037,
   -0.0007943232776597142,
   -0.011782689020037651,
   0.006948094815015793,
   0.10289479047060013,
   0.0531567819416523,
   -0.006917991675436497,
   0.0018158953171223402,
   -0.08616230636835098,
   -0.040312301367521286,
   0.041093695908784866,
   0.008172883652150631,
   0.028126629069447517,
   0.02824457362294197,
   -0.0507260337471962,
   0.03594477102160454,
   -0.11390077322721481,
   -0.049474913626909256,
   0.012679234147071838,
   0.05435965955257416,
   -0.05475328117609024,
   

In [28]:
results = collection.query(query_texts=dataset["question"][:10], n_results=1)

In [29]:
# looking at first 5 questions corresponding to the results returned by collection.query()
dataset['question'][:5]

['Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'Which vitamin is supplied from only animal source:',
 'All of the following are surgical options for morbid obesity except -',
 'Per rectum examination is not a useful test for diagnosis of',
 'Naglers reaction is shown by']

In [30]:
# Now, we will print the first 5 questions and their explanations as returned by the query()
for i, q in enumerate(dataset['question'][:5]):
    print(f"Question: {q}")
    print(f"Explanation: {results['documents'][i][0]}")
    print()

Question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma
Explanation: Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950

Question: Which vitamin is supplied from only animal source:
Explanation: Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, su